This is a start I will clean it up
TO DO

1. Load all the modules
2. read the training pool 
3. select random training set of 25 structure from the pool (can be done with np.rand) --> latter exclude these from the pool
4. Train a committee (just to check we can train 2)
5. predict on the training pool and sort max energy error
6. Then we repeat in a for loop.

In [10]:
import os
from ase.io import read, write
zundel_pool = read('/Users/shubhamdongriyal/my-drive/SabIA/ConferencesAndTalks/Brazil_july2024/tutorial/AL-tutorial-main/initial-datasets/zundel/train.extxyz', index=':')

In [11]:
import warnings
warnings.filterwarnings("ignore")
from mace.cli.run_train import main as mace_run_train_main
import sys
import logging

def train_mace(config_file_path):
    logging.getLogger().handlers.clear()
    sys.argv = ["program", "--config", config_file_path]
    mace_run_train_main()

In [12]:
print(f'Total number of structures in pool {len(zundel_pool)}')
import numpy as np

np.random.seed(0)
selected_indices = np.random.choice(len(zundel_pool), size=25, replace=False)
print(f'Selected indices: {selected_indices}')
initial_training_set = [zundel_pool[i] for i in selected_indices]

# Save the selected structures to a new file
write('initial_training_set.extxyz', initial_training_set, format='extxyz')


Total number of structures in pool 10001
Selected indices: [9395  898 2398 4737 2343 5111 5507 6452 2670 3188 1087 1819 2308 6085
 3725 8704 7052 8563 2702 6782 2931 8754 8229 4463 3032]


Hyperparameters for the committee members

In [13]:
%%writefile config/config-01.yml

model: "MACE"
num_channels: 32
max_L: 0
r_max: 4.0
name: "mace_com1"
model_dir: "MACE_models"
log_dir: "MACE_models"
checkpoints_dir: "MACE_models"
results_dir: "MACE_models"
train_file: "initial_training_set.extxyz"
energy_key: "REF_energy"
forces_key: "REF_forces"
E0s: "average"
device: cpu
batch_size: 2
max_num_epochs: 20
swa: True
seed: 1

Overwriting config/config-01.yml


In [14]:
%%writefile config/config-02.yml

model: "MACE"
num_channels: 32
max_L: 0
r_max: 4.0
name: "mace_com2"
model_dir: "MACE_models"
log_dir: "MACE_models"
checkpoints_dir: "MACE_models"
results_dir: "MACE_models"
train_file: "initial_training_set.extxyz"
energy_key: "REF_energy"
forces_key: "REF_forces"
E0s: "average"
device: cpu
batch_size: 2
max_num_epochs: 20
swa: True
seed: 2

Overwriting config/config-02.yml


In [15]:
train_mace("config/config-01.yml")
#train_mace("config/config-02.yml")

2025-05-15 07:25:05.759 INFO: ===========VERIFYING SETTINGS===========


2025-05-15 07:25:05.762 INFO: MACE version: 0.3.13
2025-05-15 07:25:05.763 INFO: Using CPU
2025-05-15 07:25:05.768 INFO: ===========LOADING INPUT DATA===========
2025-05-15 07:25:05.769 INFO: Using heads: ['Default']
2025-05-15 07:25:05.769 INFO: Using the key specifications to parse data:
2025-05-15 07:25:05.770 INFO: Default: KeySpecification(info_keys={'energy': 'REF_energy', 'stress': 'REF_stress', 'virials': 'REF_virials', 'dipole': 'dipole', 'head': 'head'}, arrays_keys={'forces': 'REF_forces', 'charges': 'REF_charges'})
2025-05-15 07:25:05.771 INFO: =============    Processing head Default     ===========
2025-05-15 07:25:05.788 INFO: Training set 1/1 [energy: 25, stress: 0, virials: 0, dipole components: 0, head: 25, forces: 25, charges: 0]
2025-05-15 07:25:05.790 INFO: Total Training set [energy: 25, stress: 0, virials: 0, dipole components: 0, head: 25, forces: 25, charges: 0]
2025-05-15 07:25:05.791 INFO: No validation set provided, splitting training data instead.
2025-05-1